# Kernel Semântico

Neste exemplo de código, irá utilizar o [Kernel Semântico](https://aka.ms/ai-agents-beginners/semantic-kernel), uma framework de IA, para criar um agente básico.

O objetivo deste exemplo é mostrar-lhe os passos que iremos utilizar mais tarde nos exemplos de código adicionais ao implementar os diferentes padrões de agentes.


## Importar os Pacotes Necessários de Python


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Criar o Cliente

Neste exemplo, vamos utilizar os [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acesso ao LLM.

O `ai_model_id` está definido como `gpt-4o-mini`. Experimente alterar o modelo para outro disponível no marketplace do GitHub Models para observar diferentes resultados.

Para utilizarmos o `Azure Inference SDK`, que é usado para o `base_url` dos GitHub Models, vamos usar o conector `OpenAIChatCompletion` dentro do Semantic Kernel. Existem também outros [conectores disponíveis](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) para usar o Semantic Kernel com outros fornecedores de modelos.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Criar o Agente

Abaixo, criamos o Agente chamado `TravelAgent`.

Neste exemplo, estamos a usar instruções muito simples. Pode alterar estas instruções para ver como o agente responde de forma diferente.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Executar o Agente

Agora podemos executar o Agente definindo uma thread do tipo `ChatHistoryAgentThread`. Quaisquer mensagens de sistema necessárias são fornecidas ao argumento de palavra-chave `messages` do método invoke_stream do agente.

Depois de definir estas mensagens, criamos um `user_inputs`, que será o que o utilizador enviará para o agente. Neste caso, definimos esta mensagem como `Plan me a sunny vacation`.

Sinta-se à vontade para alterar esta mensagem e ver como o agente responde de forma diferente.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
